In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [3]:
chars = sorted(list(set(''.join(words))))
s_to_i = {s:i+1 for i,s in enumerate(chars)}
s_to_i['.'] = 0
i_to_s = {i:s for s,i in s_to_i.items()}
vocab_size = len(i_to_s)
print(i_to_s)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = s_to_i[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [8]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability; makes exp() not overflow
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3483, grad_fn=<NegBackward0>)

In [37]:
# emb = C[Xb], [32, 3, 10]
C.shape, Xb.shape, Xb[:5]
emb[1, 1]

tensor([ 4.6827e-01, -6.5650e-01,  6.1662e-01, -6.2198e-01,  5.1007e-01,
         1.3563e+00,  2.3445e-01, -4.5585e-01, -1.3132e-03, -5.1161e-01],
       grad_fn=<SelectBackward0>)

In [39]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one


dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
dprobs = (1.0/probs) * dlogprobs

# why a manual .sum()?
# ref: https://youtu.be/q8SA3rM6ckI?list=PLXYLzZ3XzIbi4lL43O6fIU_ojuZwBO6vi&t=1440
# probs = counts * counts_sum_inv is two operations, not one
# 1. replicate the single column tensor across all rows
# 2. multiplication
# so, counts * dprobs is the derivative IF the tensors were of the same shape
# counts_sum_inv is the replicated tensor (think of node in micrograd)
# during backprop this node will be used multiple times
# therefore the gradient must be SUMMED because that's what we saw in micrograd
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

# no summation required here because it will auto broadcast
# [32, 1] * [32, 27]
dcounts = counts_sum_inv * dprobs

dcounts_sum = -(1/counts_sum**2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
dnorm_logits = counts * dcounts # same as norm_logits.exp() * dcounts
dlogits = dnorm_logits.clone() # 1.0 * dnorm_logits coz derivative is 1

# sum() again because the tensor along the single dimension [32, 1]
# is a node being used multiple times and therefore has to be summed
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)

# logits[0] = [2.1, -1.0, 3.5, 0.2]  # max is at index 2
# when gradient flows back through logit_maxes, it should only go to logits[0, 2] (the max element), not to the other elements.
# Row 5: logits[5] = [1.0, 2.0, 0.5]  # max at index 1
# one_hot for row 5 = [0, 1, 0]       # "scatter" the gradient only to position 1
# dlogit_maxes[5] = -0.03
# Result: [0, -0.03, 0]               # gradient only flows to the max element
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

# derivative of a = tanh(x) is 1 - a**2
dhpreact = (1 - h**2) * dh

dbngain = (bnraw.clone() * dhpreact).sum(0, keepdim=True)
dbnraw = (bngain.clone() * dhpreact)
dbnbias = dhpreact.sum(0, keepdim=True)
dbnvar_inv = (bndiff * dbnraw.clone()).sum(0, keepdim=True)
dbnvar = -0.5*(bnvar + 1e-5)**-1.5 * dbnvar_inv.clone()
dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar.clone()

dbndiff = (bnvar_inv * dbnraw.clone())
dbndiff += (2 * bndiff) * dbndiff2.clone()
dbnmeani = (-torch.ones_like(bnmeani) * dbndiff.clone()).sum(0, keepdim=True)
dhprebn = (torch.ones_like(hprebn) * dbndiff.clone())
dhprebn += 1/n * torch.ones_like(hprebn) * dbnmeani.clone()
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)

dC = torch.zeros(C.shape)
for i in range(Xb.shape[0]):
	for j in range(Xb.shape[1]):
		idx = Xb[i, j]
		# [i, j] is the index of the character of that batch
		# demb[i, j] is the gradient of the embeddings
		# of that particular character
		dC[idx] += demb[i, j]


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [40]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.3483283519744873 diff: -2.384185791015625e-07


In [42]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n
# -----------------

cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 6.752088665962219e-09


**Derived equation:**
```
dL/dx_i = γ(σ² + ε)^(-1/2) · [dL/dy_i - (1/m)Σ_j dL/dy_j - (1/(m-1))x̂_i · Σ_j(dL/dy_j · x̂_j)]
```

**Mapping to the code:**

Let me identify the variables:
- `bngain` = γ
- `bnvar_inv` = (σ² + ε)^(-1/2) = 1/σ
- `n` = m (batch size)
- `dhpreact` = dL/dy_i (gradient w.r.t. output)
- `bnraw` = x̂_i (normalized values)
- `dhprebn` = dL/dx_i (what we're computing)

**Rewriting your equation with code variable names:**
```
dL/dx_i = γ · (1/σ) · [dL/dy_i - (1/n)Σ dL/dy_j - (1/(n-1))x̂_i · Σ(dL/dy_j · x̂_j)]
```

**Factor out common terms:**

Multiply through by n inside the brackets:
```
dL/dx_i = (γ/σ)/n · [n·dL/dy_i - Σ dL/dy_j - (n/(n-1))x̂_i · Σ(dL/dy_j · x̂_j)]
```

**This matches the code:**
```python
dhprebn = bngain * bnvar_inv / n * (
    n * dhpreact                           # n·dL/dy_i
    - dhpreact.sum(0)                      # -Σ dL/dy_j
    - n/(n-1) * bnraw * (dhpreact*bnraw).sum(0)  # -(n/(n-1))x̂_i·Σ(dL/dy_j·x̂_j)
)
```

The key transformation is **factoring out γ/(σ·n)** from all three terms, which allows the computation to be more efficient!

In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n - 1)*bnraw*(dhpreact*bnraw).sum(0))
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [47]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
	p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

	# minibatch construct
	ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
	Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

	# forward pass
	emb = C[Xb] # embed the characters into vectors
	embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
	# Linear layer
	hprebn = embcat @ W1 + b1 # hidden layer pre-activation
	# BatchNorm layer
	# -------------------------------------------------------------
	bnmean = hprebn.mean(0, keepdim=True)
	bnvar = hprebn.var(0, keepdim=True, unbiased=True)
	bnvar_inv = (bnvar + 1e-5)**-0.5
	bnraw = (hprebn - bnmean) * bnvar_inv
	hpreact = bngain * bnraw + bnbias
	# -------------------------------------------------------------
	# Non-linearity
	h = torch.tanh(hpreact) # hidden layer
	logits = h @ W2 + b2 # output layer
	loss = F.cross_entropy(logits, Yb) # loss function

	# backward pass
	for p in parameters:
		p.grad = None
	# loss.backward() # use this for correctness comparisons, delete it later!

	# manual backprop! #swole_doge_meme
	# -----------------
	# YOUR CODE HERE :)
	dlogits = F.softmax(logits, 1)
	dlogits[range(n), Yb] -= 1
	dlogits /= n

	# second layer backprop
	dh = dlogits @ W2.T
	dW2 = h.T @ dlogits
	db2 = dlogits.sum(0)

	# tanh
	dhpreact = (1 - h**2) * dh

	# batchnorm backprop
	dbngain = (bnraw.clone() * dhpreact).sum(0, keepdim=True)
	dbnbias = dhpreact.sum(0, keepdim=True)
	dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n - 1)*bnraw*(dhpreact*bnraw).sum(0))

	# first layer
	dembcat = dhprebn @ W1.T
	dW1 = embcat.T @ dhprebn
	db1 = dhprebn.sum(0)

	# embedding
	demb = dembcat.view(emb.shape)
	dC = torch.zeros(C.shape)
	for k in range(Xb.shape[0]):
		for j in range(Xb.shape[1]):
			idx = Xb[k, j]
			# [k, j] is the index of the character of that batch
			# demb[k, j] is the gradient of the embeddings
			# of that particular character
			dC[idx] += demb[k, j]

	grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
	# -----------------

	# update
	lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
	for p, grad in zip(parameters, grads):
		# p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
		p.data += -lr * grad # new way of swole doge TODO: enable

	# track stats
	if i % 10000 == 0: # print every once in a while
		print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
		lossi.append(loss.log10().item())

	# if i >= 100: # TODO: delete early breaking when you're ready to train the full net
	# 	break

12297
      0/ 200000: 3.7575
  10000/ 200000: 2.1793
  20000/ 200000: 2.3541
  30000/ 200000: 2.4285
  40000/ 200000: 1.9297
  50000/ 200000: 2.3755
  60000/ 200000: 2.3591
  70000/ 200000: 2.0580
  80000/ 200000: 2.3322
  90000/ 200000: 2.0024
 100000/ 200000: 1.9579
 110000/ 200000: 2.3269
 120000/ 200000: 2.0456
 130000/ 200000: 2.4274
 140000/ 200000: 2.3297
 150000/ 200000: 2.1306
 160000/ 200000: 1.9339
 170000/ 200000: 1.8115
 180000/ 200000: 2.0385
 190000/ 200000: 1.9122


In [46]:
# useful for checking your gradients
for param ,grad in zip(parameters, grads):
	cmp(str(tuple(param.shape)), grad, param)

(27, 10)        | exact: False | approximate: True  | maxdiff: 1.5832483768463135e-08
(30, 200)       | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
(200,)          | exact: False | approximate: True  | maxdiff: 3.4924596548080444e-09
(200, 27)       | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
(27,)           | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
(1, 200)        | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
(1, 200)        | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09


In [48]:
# calibrate the batch norm at the end of training

with torch.no_grad():
	# pass the training set through
	emb = C[Xtr]
	embcat = emb.view(emb.shape[0], -1)
	hpreact = embcat @ W1 + b1
	# measure the mean/std over the entire training set
	bnmean = hpreact.mean(0, keepdim=True)
	bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [49]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
	x,y = {
		'train': (Xtr, Ytr),
		'val': (Xdev, Ydev),
		'test': (Xte, Yte),
	}[split]
	emb = C[x] # (N, block_size, n_embd)
	embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
	hpreact = embcat @ W1 + b1
	hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
	h = torch.tanh(hpreact) # (N, n_hidden)
	logits = h @ W2 + b2 # (N, vocab_size)
	loss = F.cross_entropy(logits, y)
	print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0725653171539307
val 2.111285924911499


In [50]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

	out = []
	context = [0] * block_size # initialize with all ...
	while True:
		# forward pass
		emb = C[torch.tensor([context])] # (1,block_size,d)
		embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
		hpreact = embcat @ W1 + b1
		hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
		h = torch.tanh(hpreact) # (N, n_hidden)
		logits = h @ W2 + b2 # (N, vocab_size)
		# sample
		probs = F.softmax(logits, dim=1)
		ix = torch.multinomial(probs, num_samples=1, generator=g).item()
		context = context[1:] + [ix]
		out.append(ix)
		if ix == 0:
			break

	print(''.join(i_to_s[i] for i in out))

carmah.
amille.
khyimleigentleenanden.
jazonel.
delynn.
jareei.
ner.
kiah.
maiivia.
leigh.
ham.
join.
quint.
shoen.
alianni.
wavero.
dearisia.
kaellinslee.
dae.
iia.
